In [22]:
"""""!pip install numpy
!pip install pandas 
!pip install time
!pip install pprint
!pip install collections
!pip install matplotlib
!pip install seaborn 
!pip install sklearn
!pip install warnings"""""

'""!pip install numpy\n!pip install pandas \n!pip install time\n!pip install pprint\n!pip install collections\n!pip install matplotlib\n!pip install seaborn \n!pip install sklearn\n!pip install warnings'

In [49]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_colwidth', -1)
import os
from time import time
from pprint import pprint
import collections

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")
sns.set(font_scale=1.3)

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib

import warnings
warnings.filterwarnings('ignore')

np.random.seed(37)

In [288]:
df_model = pd.read_pickle('../../data/final/NotIot_clean_EN_tweets_polarity.p')

In [289]:
df_model.head()

,count_words,count_mentions,count_hashtags,count_capital_words,count_excl_quest_marks,count_urls,count_emojis,sentiment,origin_text,clean_text,lex_anger,lex_sadness,lex_joy,lex_fear
0,22,5,2,1,1,0,0,positive,Happy to share my new video preview with some of my local fav's!\n#compose #excitement @CBCMusic @Musicyyc @CJSW @BreakOutWest @HifiClub,happy share new video preview local fav compose excitement,0.0000,0.000000,0.238667,0.000000
1,14,5,0,0,0,0,0,positive,@FieldYates @MatthewBerryTMR @Stephania_ESPN @MikeClayNFL @FrankCaliendo goddamn...the 'celebrity' draft at the end was classic.,goddamn celebrity draft end classic,0.0052,0.000000,0.068400,0.000000
2,11,0,0,0,0,0,0,negative,Dreams dashed and divided like million stars in the night sky.,dreams dashed divided like million stars night sky,0.0410,0.080125,0.120625,0.037125
3,17,0,0,0,0,0,1,positive,Accept the challenges so that you can literally even feel the exhilaration of victory.' -- George S. Patton 🐶,accept challenges literally even feel exhilaration victory george patton dogface,0.0364,0.000000,0.162500,0.028100
4,15,1,1,0,0,0,0,positive,Happy birthday to ma brother from another mother @Official1Cedi dis year More blessing #Break itz,happy birthday brother another mother dis year blessing break,0.0000,0.001778,0.321222,0.000000


In [290]:
X_train, X_test, y_train, y_test = train_test_split(df_model.drop('sentiment', axis=1), df_model.sentiment, test_size=0.1, random_state=37)

In [291]:
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X, **transform_params):
        return X[self.cols]

    def fit(self, X, y=None, **fit_params):
        return self

In [292]:
def grid_vect(clf, parameters_clf, X_train, X_test, parameters_text=None, vect=None):
    
      
    textcountscols = ['count_capital_words','count_emojis'
                      ,'count_excl_quest_marks','count_hashtags','count_mentions','count_urls',
                      'count_words']
    
    lexiccols = ['lex_anger','lex_sadness','lex_joy','lex_fear']#,'lex_surprise','lex_disgust','lex_neutre']
    

    features = FeatureUnion([ ('textcounts', ColumnExtractor(cols=textcountscols)),
                            ('lexiccols', ColumnExtractor(cols=lexiccols)),
                        ('pipe', Pipeline([('cleantext', ColumnExtractor(cols='clean_text')), ('vect', vect)]))]
                            , n_jobs=-1)
    print(features)
    pipeline = Pipeline([
        ('features', features)
        , ('clf', clf)
    ])
    
    # Join the parameters dictionaries together
    parameters = dict()
    if parameters_text:
        parameters.update(parameters_text)
    parameters.update(parameters_clf)

    # initiate gridsearchCV with parameters and pipline
    grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, cv=5)
    
    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(parameters)

    t0 = time()
    grid_search.fit(X_train, y_train)
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best CV score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
        
    print("Test score with best_estimator_: %0.3f" % grid_search.best_estimator_.score(X_test, y_test))
    print("\n")
    print("Classification Report Test Data")
    print(classification_report(y_test, grid_search.best_estimator_.predict(X_test)))

    print("all results")
    means = grid_search.cv_results_['mean_test_score']
    stds = grid_search.cv_results_['std_test_score']
    params = grid_search.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
    
    return grid_search

In [293]:
# Parameter grid settings for the vectorizers (Count and TFIDF)
parameters_vect = {
    'features__pipe__vect__max_df': (0.25, 0.5, 0.75),
    'features__pipe__vect__ngram_range': ((1, 1), (1, 2)),
    'features__pipe__vect__min_df': (1,2)
}

# Parameter grid settings for SVM
parameters_svm = {'clf__kernel':('linear', 'rbf'), 'clf__C':(1,0.5,0.75),'clf__gamma': (1,2)}

In [294]:
from sklearn.svm import SVC
svm = SVC()
svm.get_params().keys()

dict_keys(['C', 'cache_size', 'class_weight', 'coef0', 'decision_function_shape', 'degree', 'gamma', 'kernel', 'max_iter', 'probability', 'random_state', 'shrinking', 'tol', 'verbose'])

In [295]:
countvect = CountVectorizer()

In [296]:
import sys
#sys.stdout = open("output_SVMModel_sentimentEN_features_count.txt", "w")

In [297]:
# SVM
best_mnb_countvect = grid_vect(svm, parameters_svm, X_train, X_test, parameters_text=parameters_vect, vect=countvect)

FeatureUnion(n_jobs=-1,
       transformer_list=[('textcounts', ColumnExtractor(cols=['count_capital_words', 'count_emojis', 'count_excl_quest_marks', 'count_hashtags', 'count_mentions', 'count_urls', 'count_words'])), ('lexiccols', ColumnExtractor(cols=['lex_anger', 'lex_sadness', 'lex_joy', 'lex_fear'])), ('pipe', Pipeline(memo...strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None))]))],
       transformer_weights=None)
Performing grid search...
pipeline: ['features', 'clf']
parameters:
{'clf__C': (1, 0.5, 0.75),
 'clf__gamma': (1, 2),
 'clf__kernel': ('linear', 'rbf'),
 'features__pipe__vect__max_df': (0.25, 0.5, 0.75),
 'features__pipe__vect__min_df': (1, 2),
 'features__pipe__vect__ngram_range': ((1, 1), (1, 2))}
Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 19.5min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 40.7min
[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed: 64.5min finished


done in 3933.239s

Best CV score: 0.782
Best parameters set:
	clf__C: 0.75
	clf__gamma: 1
	clf__kernel: 'linear'
	features__pipe__vect__max_df: 0.25
	features__pipe__vect__min_df: 1
	features__pipe__vect__ngram_range: (1, 2)
Test score with best_estimator_: 0.806


Classification Report Test Data
              precision    recall  f1-score   support

    negative       0.83      0.85      0.84       441
    positive       0.77      0.74      0.76       303

   micro avg       0.81      0.81      0.81       744
   macro avg       0.80      0.80      0.80       744
weighted avg       0.81      0.81      0.81       744

all results
0.773080 (0.014538) with: {'clf__C': 1, 'clf__gamma': 1, 'clf__kernel': 'linear', 'features__pipe__vect__max_df': 0.25, 'features__pipe__vect__min_df': 1, 'features__pipe__vect__ngram_range': (1, 1)}
0.781147 (0.016694) with: {'clf__C': 1, 'clf__gamma': 1, 'clf__kernel': 'linear', 'features__pipe__vect__max_df': 0.25, 'features__pipe__vect__min_df': 1, 'feature

In [298]:
tfidfvect = TfidfVectorizer()

In [299]:
#sys.stdout = open("output_SVMModel_sentimentEN_features_tfidf.txt", "w")

In [300]:
best_mnb_tfidf = grid_vect(svm, parameters_svm, X_train, X_test, parameters_text=parameters_vect, vect=tfidfvect)

FeatureUnion(n_jobs=-1,
       transformer_list=[('textcounts', ColumnExtractor(cols=['count_capital_words', 'count_emojis', 'count_excl_quest_marks', 'count_hashtags', 'count_mentions', 'count_urls', 'count_words'])), ('lexiccols', ColumnExtractor(cols=['lex_anger', 'lex_sadness', 'lex_joy', 'lex_fear'])), ('pipe', Pipeline(memo...      token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None))]))],
       transformer_weights=None)
Performing grid search...
pipeline: ['features', 'clf']
parameters:
{'clf__C': (1, 0.5, 0.75),
 'clf__gamma': (1, 2),
 'clf__kernel': ('linear', 'rbf'),
 'features__pipe__vect__max_df': (0.25, 0.5, 0.75),
 'features__pipe__vect__min_df': (1, 2),
 'features__pipe__vect__ngram_range': ((1, 1), (1, 2))}
Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 18.6min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 38.2min
[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed: 61.9min finished


done in 3744.593s

Best CV score: 0.804
Best parameters set:
	clf__C: 0.5
	clf__gamma: 1
	clf__kernel: 'linear'
	features__pipe__vect__max_df: 0.25
	features__pipe__vect__min_df: 1
	features__pipe__vect__ngram_range: (1, 1)
Test score with best_estimator_: 0.819


Classification Report Test Data
              precision    recall  f1-score   support

    negative       0.82      0.88      0.85       441
    positive       0.81      0.72      0.76       303

   micro avg       0.82      0.82      0.82       744
   macro avg       0.82      0.80      0.81       744
weighted avg       0.82      0.82      0.82       744

all results
0.800269 (0.009261) with: {'clf__C': 1, 'clf__gamma': 1, 'clf__kernel': 'linear', 'features__pipe__vect__max_df': 0.25, 'features__pipe__vect__min_df': 1, 'features__pipe__vect__ngram_range': (1, 1)}
0.796086 (0.010369) with: {'clf__C': 1, 'clf__gamma': 1, 'clf__kernel': 'linear', 'features__pipe__vect__max_df': 0.25, 'features__pipe__vect__min_df': 1, 'features

In [13]:
#max_df: 
#min_df: 
#ngram_range: 
#clf__C :
#textcountscols = ['count_capital_words','count_emojis','count_excl_quest_marks','count_hashtags'
#                      ,'count_mentions','count_urls','count_words']
#    az
#features = FeatureUnion([('textcounts', ColumnExtractor(cols=textcountscols))
#                         , ('pipe', Pipeline([('cleantext', ColumnExtractor(cols='clean_text'))
#                                              , ('vect', CountVectorizer(max_df=0.25, min_df=1, ngram_range=(1,2)))]))]
#                       , n_jobs=-1)
#
#pipeline = Pipeline([
#    ('features', features)
#    , ('clf', SVC(C=0.75))
#])

#best_model = pipeline.fit(df_model.drop('sentiment', axis=1), df_model.sentiment)

In [30]:
#!jupyter nbconvert --to script SVMModel_sentimentEN_features.ipynb